In [2]:

from google.colab import drive
import sqlite3
import csv
import json

# Mount Google Drive
drive.mount('/content/gdrive')

# Function to connect to SQLite database
def connect_to_sqlite(database_path):
    try:
      #connect to SQLite database
        conn = sqlite3.connect(database_path)
        return conn
    except sqlite3.Error as e:
      #print error if connection fails
        print(f"Error connecting to SQLite database: {e}")
        return None

# Function to create database tables
def create_tables(cursor):
    try:
      #create album table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Album (
                AlbumId INTEGER PRIMARY KEY,
                Title TEXT,
                ArtistId INTEGER,
                FOREIGN KEY (ArtistId) REFERENCES Artist(ArtistId)
            )
        """)
        #create artist table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Artist (
                ArtistId INTEGER PRIMARY KEY,
                Name TEXT,
                Followers INTEGER,
                Genres TEXT,
                Popularity INTEGER
            )
        """)
        #create track table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Track (
                TrackId INTEGER PRIMARY KEY,
                Name TEXT,
                Popularity INTEGER,
                Artists TEXT,
                ArtistId INTEGER,
                ReleaseDate TEXT,
                FOREIGN KEY (ArtistId) REFERENCES Artist(ArtistId)
            )
        """)
        #create recommendation table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Recommendation (
                ArtistName TEXT PRIMARY KEY,
                RecommendedArtists TEXT
            )
        """)
    except sqlite3.Error as e:
      #print error if table creation fails
        print(f"Error creating tables: {e}")

# Function to extract data from CSV file
def extract_from_csv(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
          #read csv file as dictionary
            reader = csv.DictReader(file)
            return [dict(row) for row in reader]
    except Exception as e:
      #print error if extraction fails
        print(f"Error extracting data from CSV: {e}")
        return []

# Function to extract data from JSON file
def extract_from_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
          #load JSON data
            return json.load(file)
    except Exception as e:
      #print error if extraction fails
        print(f"Error extracting data from JSON: {e}")
        return {}

# Function to transform CSV data
def transform_csv_data(data):
    transformed_data = []
    for row in data:
        # extract data and handle potential missing values
        if 'artists' not in row:
            continue

        # Extract and transform data
        track_id = row.get('id', '')
        name = row.get('name', '')
        popularity = int(row.get('popularity', 0))
        artists = row.get('artists', '').split(',')  # Split artists into a list
        release_date = row.get('release_date', '')

    return transformed_data

# Function to transform JSON data
def transform_json_data(data):
    transformed_data = []
    for artist, recommended_artists in data.items():
      # Join recommended artists into a comma-separated string
        transformed_row = [artist, ','.join(recommended_artists)]
        transformed_data.append(transformed_row)
    return transformed_data

# Function to load data into SQLite table
def load_into_sqlite(cursor, table_name, data):
    try:
        if table_name == 'Artist':
            cursor.executemany("""
                INSERT INTO Artist (ArtistId, Name, Followers, Genres, Popularity)
                VALUES (?, ?, ?, ?, ?)
            """, data)
        elif table_name == 'Track':
            cursor.executemany("""
                INSERT INTO Track (TrackId, Name, Popularity, Artists, ArtistId, ReleaseDate)
                VALUES (?, ?, ?, ?, ?, ?)
            """, data)
        elif table_name == 'Recommendation':
            for row in data:
                artist_name = row[0]
                recommended_artists = row[1]
                cursor.execute("INSERT OR IGNORE INTO Recommendation (ArtistName, RecommendedArtists) VALUES (?, ?)", (artist_name, recommended_artists))
    except sqlite3.Error as e:
      #print error if loading fails
        print(f"Error loading data into SQLite: {e}")
        return False
    return True

# Main function
def main():
    # Connect to SQLite database
    sqlite_conn = connect_to_sqlite('/content/gdrive/My Drive/music.db')
    if sqlite_conn:
        sqlite_cursor = sqlite_conn.cursor()
        create_tables(sqlite_cursor)

        # Extract data from CSV files
        artists_csv_data = extract_from_csv("/content/gdrive/My Drive/artists.csv")
        tracks_csv_data = extract_from_csv("/content/gdrive/My Drive/tracks.csv")

        # Transform CSV data
        transformed_artists_data = transform_csv_data(artists_csv_data)
        transformed_tracks_data = transform_csv_data(tracks_csv_data)

        # Load data into SQLite tables
        load_into_sqlite(sqlite_cursor, 'Artist', transformed_artists_data)
        load_into_sqlite(sqlite_cursor, 'Track', transformed_tracks_data)

        # Extract data from JSON file
        dict_artists_json_data = extract_from_json("/content/gdrive/My Drive/dict_artists.json")

        # Transform JSON data
        transformed_dict_artists_data = transform_json_data(dict_artists_json_data)

        # Load data into SQLite table
        load_into_sqlite(sqlite_cursor, 'Recommendation', transformed_dict_artists_data)

        # Commit changes and close connection
        sqlite_conn.commit()
        sqlite_cursor.close()
        sqlite_conn.close()

if __name__ == "__main__":
    main()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
